In [1]:
import pandas as pd
import threading
from concurrent.futures import ThreadPoolExecutor
import time

In [2]:
time1 = time.time()

df = pd.read_excel(r"C:\Users\Administrator\Desktop\test.xlsx")
print(df)

temps = []
indexs = []

urls = df.loc[:, "URL"].tolist()
for i in range(df.shape[0]):
    temps.append(0)
    indexs.append(i)
    
print(temps)
print(indexs)
print(urls)

                                      URL
0    https://www.amazon.com/dp/B09XT7PM39
1    https://www.amazon.com/dp/B0DS5Y6KK2
2    https://www.amazon.com/dp/B0BVLL91T8
3    https://www.amazon.com/dp/B0CWP42BQT
4    https://www.amazon.com/dp/B0DGX9SJYH
..                                    ...
463  https://www.amazon.com/dp/B0CHMM2TY3
464  https://www.amazon.com/dp/B0CSXYLGW5
465  https://www.amazon.com/dp/B0BFJBDQ7W
466  https://www.amazon.com/dp/B018YB1UA0
467  https://www.amazon.com/dp/B0CJ4L43NQ

[468 rows x 1 columns]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [3]:
def function(params):
    import re
    from datetime import datetime
    from selenium import webdriver
    from selenium.webdriver.edge.service import Service as EdgeService
    from selenium.webdriver.common.by import By
    from amazoncaptcha import AmazonCaptcha
    
    service = EdgeService(executable_path=r"C:\Users\Administrator\msedgedriver.exe")
    options = webdriver.EdgeOptions()
    # 使用headless无界面浏览器模式和禁用gpu加速
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Edge(service=service, options=options)
    
    url,index = params
    driver.get(url)
    
    try:
        xpath1 = '/html/body/div/div[1]/div[3]/div/div/form/div[1]/div/div/div[1]/img'
        b = driver.find_element(By.XPATH,xpath1).get_attribute('src')
        captcha = AmazonCaptcha.fromlink(b) # 识别链接或路径表示的验证码
        solution = captcha.solve()#识别后返回的结果，字符型
        # print(solution)
        driver.find_element(By.XPATH,'//*[@id="captchacharacters"]').send_keys(solution) # 输入验证码
        driver.find_element(By.XPATH,'/html/body/div/div[1]/div[3]/div/div/form/div[2]/div/span/span/button').click()
    except:
        pass
        
    driver.refresh()
    
    try:
        date_first_available = driver.find_element(By.XPATH, '//*[@id="productDetails_detailBullets_sections1"]')
        # 获取 <td> 元素的文本内容
        text = date_first_available.text
        # 使用正则表达式提取日期信息
        pattern = r'Date First Available (\w+ \d{1,2}, \d{4})'
        match = re.search(pattern, text)
        if match:
            # 提取到的日期字符串
            date_str = match.group(1)
            # 将日期字符串转换为 datetime 对象
            date_obj = datetime.strptime(date_str, '%B %d, %Y')
            # 将 datetime 对象格式化为 yyyy-mm-dd 格式
            formatted_date = date_obj.strftime('%Y-%m-%d')
            print(formatted_date)
        else:
            print("未找到日期信息")
    except:
        formatted_date = ""
    global temps
    temps[index] = formatted_date
    print(temps)
    driver.quit()

In [4]:
# 固定线程数量为 3
num_threads = 5
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # 将参数封装为元组
    param_tuples = [(url, index) for url, index in zip(urls, indexs)]
    print(param_tuples)
    results = executor.map(function, param_tuples)

# 可以在这里处理任务的返回结果
for result in results:
    pass

print("所有任务已完成")

[('https://www.amazon.com/dp/B09XT7PM39', 0), ('https://www.amazon.com/dp/B0DS5Y6KK2', 1), ('https://www.amazon.com/dp/B0BVLL91T8', 2), ('https://www.amazon.com/dp/B0CWP42BQT', 3), ('https://www.amazon.com/dp/B0DGX9SJYH', 4), ('https://www.amazon.com/dp/B07CZ6WDB5', 5), ('https://www.amazon.com/dp/B0B6J268WY', 6), ('https://www.amazon.com/dp/B0BVLKFFHN', 7), ('https://www.amazon.com/dp/B0B6TZ4F2L', 8), ('https://www.amazon.com/dp/B0C246GV9X', 9), ('https://www.amazon.com/dp/B0BDQXHN8Z', 10), ('https://www.amazon.com/dp/B07T8GX584', 11), ('https://www.amazon.com/dp/B0B8SMGLSZ', 12), ('https://www.amazon.com/dp/B082ZZ2NR8', 13), ('https://www.amazon.com/dp/B0D9QFLS48', 14), ('https://www.amazon.com/dp/B0B8SMGLSZ', 15), ('https://www.amazon.com/dp/B0D9QGCT1N', 16), ('https://www.amazon.com/dp/B0DG4MM9KB', 17), ('https://www.amazon.com/dp/B08CB138TS', 18), ('https://www.amazon.com/dp/B0BYVNHY5T', 19), ('https://www.amazon.com/dp/B0BDQGBQT4', 20), ('https://www.amazon.com/dp/B0D8KGP7ZM', 21

In [5]:
print(temps)
df["ins"] = temps
time2 = time.time()
print(abs(time2-time1))

['2018-03-30', '2025-01-03', '2023-02-13', '2024-02-28', '2024-09-13', '2018-08-24', '2022-07-14', '2023-02-13', '2020-07-10', '2023-04-11', '2022-09-09', '2019-06-21', '2022-08-05', '2019-05-28', '2024-07-19', '2022-08-05', '2024-07-19', '2024-09-05', '2020-07-03', '2023-03-18', '2022-09-09', '2024-07-02', '2023-05-08', '2024-04-24', '2023-12-12', '2023-05-08', '2023-12-12', '2022-09-13', '2023-03-07', '2015-01-10', '2022-09-09', '2023-03-07', '2022-07-15', '2023-08-09', '2023-02-14', '2018-03-30', '2024-01-19', '2023-02-13', '2023-11-21', '2023-02-21', '2022-08-28', '2024-01-19', '2023-12-07', '2023-03-28', '2021-02-26', '2021-12-01', '2023-04-24', '2023-09-21', '2024-03-27', '2023-03-21', '2024-05-19', '2024-03-27', '2022-09-12', '2018-12-29', '2023-06-17', '2023-03-21', '2024-07-24', '2024-04-07', '2024-05-28', '2021-12-26', '2018-03-30', '2022-01-16', '2024-02-29', '2023-12-07', '2018-07-02', '2022-09-12', '2024-02-28', '2024-02-28', '2022-08-28', '2023-09-08', '2020-09-25', '2023

In [6]:
df.to_excel("test.xlsx")